In [341]:
import pandas as pd
import numpy as np
import json
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
from dotenv import load_dotenv
import PyPDF2
import os

In [342]:
load_dotenv()
open_api_key=os.environ.get('OPENAI_API_KEY')
open_api_key='sk-proj-WRpvpdHvkooHFU8zCNXtT3BlbkFJsxuxYAQaxjINuDsOPS0J'

In [343]:
llm=ChatOpenAI(openai_api_key=open_api_key,model='gpt-3.5-turbo',temperature=0.7)

In [344]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000021A0467A5F0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000021A046792D0>, openai_api_key='sk-proj-WRpvpdHvkooHFU8zCNXtT3BlbkFJsxuxYAQaxjINuDsOPS0J', openai_proxy='')

In [345]:
with open(r'C:\Users\HP\OneDrive\Desktop\desktop\generativeAI\openai_mcq_gen\Response.json') as f:
    Response_JSON=json.load(f)

In [346]:
print(Response_JSON)

{'1': {'no': '1', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'no': '2', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'no': '3', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [347]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [348]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=TEMPLATE
)

In [349]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [350]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [351]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [352]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [353]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],output_variables=["quiz", "review"], verbose=True,)

In [354]:
with open(r'C:\Users\HP\OneDrive\Desktop\desktop\generativeAI\openai_mcq_gen\data.txt') as file:
    TEXT=file.read()

In [355]:
print(TEXT)

Photosynthesis, the plantâ€™s magical cooking, unfolds in tiny kitchens called chloroplasts. These mini-factories, wrapped in a double layer, hold a green chef called chlorophyll. Inside, there are flat sacs called thylakoids, forming stacks named grana in a clear space called stroma.Chlorophyll, the chefâ€™s assistant, loves catching red and blue light, giving plants their green color. This kitchen duo transforms air and water into yummy carbohydrates, and as a bonus, they serve up oxygen. Itâ€™s like a dance â€“ sunlight, colors, and plant kitchens working together to make lifeâ€™s essential recipe, the enchanting process of photosynthesis.


In [356]:
Text=TEXT
NUMBER=5
SUBJECT='Biology'
TONE='Simple'
RESPONSE_JSON=Response_JSON

In [357]:
Response_JSON

{'1': {'no': '1',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'no': '2',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'no': '3',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [358]:
with  get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {

        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "RESPONSE_JSON": json.dumps(RESPONSE_JSON)

        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Photosynthesis, the plantâ€™s magical cooking, unfolds in tiny kitchens called chloroplasts. These mini-factories, wrapped in a double layer, hold a green chef called chlorophyll. Inside, there are flat sacs called thylakoids, forming stacks named grana in a clear space called stroma.Chlorophyll, the chefâ€™s assistant, loves catching red and blue light, giving plants their green color. This kitchen duo transforms air and water into yummy carbohydrates, and as a bonus, they serve up oxygen. Itâ€™s like a dance â€“ sunlight, colors, and plant kitchens working together to make lifeâ€™s essential recipe, the enchanting process of photosynthesis.
You are an expert MCQ maker. Given the above text, it is your job to create a quiz  of 5 multiple choice questions for Biology students in Simple tone. 
Make sure the questions are not repeated and check all the questions to be conforming th

In [359]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1293
Prompt Tokens:888
Completion Tokens:405
Total Cost:0.002142


In [360]:
response

{'text': 'Photosynthesis, the plantâ€™s magical cooking, unfolds in tiny kitchens called chloroplasts. These mini-factories, wrapped in a double layer, hold a green chef called chlorophyll. Inside, there are flat sacs called thylakoids, forming stacks named grana in a clear space called stroma.Chlorophyll, the chefâ€™s assistant, loves catching red and blue light, giving plants their green color. This kitchen duo transforms air and water into yummy carbohydrates, and as a bonus, they serve up oxygen. Itâ€™s like a dance â€“ sunlight, colors, and plant kitchens working together to make lifeâ€™s essential recipe, the enchanting process of photosynthesis.',
 'number': 5,
 'subject': 'Biology',
 'tone': 'Simple',
 'RESPONSE_JSON': '{"1": {"no": "1", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"no": "2", "mcq": "multiple choice questions", "options": {"a": "choice here", 

In [362]:
quiz=response.get("quiz")
quiz=json.loads(quiz)
print(quiz)

{'1': {'no': '1', 'mcq': 'What is the name of the tiny kitchens where photosynthesis takes place in plants?', 'options': {'a': 'Mitochondria', 'b': 'Chloroplasts', 'c': 'Nucleus', 'd': 'Endoplasmic Reticulum'}, 'correct': 'b'}, '2': {'no': '2', 'mcq': 'What is the name of the green chef inside the chloroplasts?', 'options': {'a': 'Carotenoid', 'b': 'Chlorophyll', 'c': 'Xanthophyll', 'd': 'Phycobilin'}, 'correct': 'b'}, '3': {'no': '3', 'mcq': 'What is the name of the flat sacs inside the chloroplasts where photosynthesis occurs?', 'options': {'a': 'Thylakoids', 'b': 'Stroma', 'c': 'Grana', 'd': 'Chlorophyll'}, 'correct': 'a'}, '4': {'no': '4', 'mcq': 'What is the main function of chlorophyll in photosynthesis?', 'options': {'a': 'Absorb red and blue light', 'b': 'Produce oxygen', 'c': 'Store energy', 'd': 'Break down carbohydrates'}, 'correct': 'a'}, '5': {'no': '5', 'mcq': 'What are the end products of photosynthesis?', 'options': {'a': 'Carbon dioxide and water', 'b': 'Oxygen and glu

In [364]:
quiz_table_data=[]
for key,value in quiz.items():
    mcq=value['mcq']
    options='|'.join(
        [
            f'{option}:{option_value}'
            for option,option_value in value['options'].items()
        ]
    )
    correct=value['correct']
    quiz_table_data.append({'MCQ':mcq,'Choices':options,'Correct':correct})

In [365]:
quiz_table_data

[{'MCQ': 'What is the name of the tiny kitchens where photosynthesis takes place in plants?',
  'Choices': 'a:Mitochondria|b:Chloroplasts|c:Nucleus|d:Endoplasmic Reticulum',
  'Correct': 'b'},
 {'MCQ': 'What is the name of the green chef inside the chloroplasts?',
  'Choices': 'a:Carotenoid|b:Chlorophyll|c:Xanthophyll|d:Phycobilin',
  'Correct': 'b'},
 {'MCQ': 'What is the name of the flat sacs inside the chloroplasts where photosynthesis occurs?',
  'Choices': 'a:Thylakoids|b:Stroma|c:Grana|d:Chlorophyll',
  'Correct': 'a'},
 {'MCQ': 'What is the main function of chlorophyll in photosynthesis?',
  'Choices': 'a:Absorb red and blue light|b:Produce oxygen|c:Store energy|d:Break down carbohydrates',
  'Correct': 'a'},
 {'MCQ': 'What are the end products of photosynthesis?',
  'Choices': 'a:Carbon dioxide and water|b:Oxygen and glucose|c:Nitrogen and hydrogen|d:Methane and ATP',
  'Correct': 'b'}]

In [366]:
df=pd.DataFrame(quiz_table_data)

In [374]:
df.to_csv('mcq_bits.csv',index=False)